In [1]:
from google.colab import drive
import jsondrive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import numpy as np
from sklearn.metrics import confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn


In [ ]:


ROOT_PATH = "/content/drive/MyDrive/uni/nlp/nlp2024-hw1-b"



In [ ]:


class HaSpeeDe_Dataset(Dataset):








